In [69]:
import matplotlib.pyplot as plt
import matplotlib
import mne
import pandas as pd
import numpy as np

In [70]:
#UTILITIES

"""
Filters the trigger signals so that only the first value of each group of triggers that is close in time is retained. 
Necessary because the triggers are analog so each real trigger results in a string of triggers being detected.

Trig array: the array straight out of mne.find_events
Threshold: the minimum difference in the first column between the previous and current row for the current row to be retained 
""" 
def clean_triggers(trig_array, threshold = 100):
    #setting threshold for how much time has to pass between each trigger
    diff_time = np.diff(trig_array[:, 0])
    indices = np.where(diff_time > threshold)[0]
    indices = np.concatenate(([0], indices + 1))
    return trig_array[indices]


"""
Concatenates different sections of the EEG experiment. It does the same thing as Epochs sort of, but you end up with a raw object instead of epoch
Useful for gathering all the recordings of muted and unmuted sections of the motor experiment. 

raw: the original raw file straight out of mne.io.read_raw_bdf
events: the events array of interest, like tc_mute
segment_dur: duration of the segment. For muted segments it's 30 seconds, for unmuted it's 10


Note: the time axis will still be continuous even if the data has been chopped up
"""
def concat_uniform(raw, events, segment_dur):
    segments = []
    for time in events[:,0]:
        print(time)
        segment_start = time/fs #because crop() uses seconds and not samples
        segment_end = segment_start+segment_dur

        #make sure the max length of eeg is not exceeded
        if segment_end > raw.times.max():
            segment_end = raw.times.max()

        segment = raw.copy().crop(tmin = segment_start, tmax = segment_end)
        segments.append(segment)

    return mne.io.concatenate_raws(segments)


def concat_nonuniform(raw, events1, events2):
    pass


In [71]:
#import BDF and setting important parameters
raw = mne.io.read_raw_bdf('motor_trig_test.bdf')

fs = raw.info['sfreq']
#raw.info.ch_names

Extracting EDF parameters from /Users/cindyzhang/Documents/M2/AM-piano/EEG/trig_tests/motor_trig_test.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


In [72]:
#loading the raw events
#the stim channel is called 'Status' 
events = mne.find_events(raw, stim_channel='Status', shortest_event=1) #raises exception if shortest event is default 2...?

Trigger channel has a non-zero initial value of 130816 (consider using initial_event=True to detect this event)
3964 events found
Event IDs: [65282 65284 65288 65290]


In [73]:
raw.info['sfreq']

2048.0

In [74]:
#sorting events into different IDs

#65282 = trig 2 (keystrokes)
#65284 = trig 3 (trig-mute in motor test)
#65288 = trig 4 (trig-unmute in motor test)
#not sure what's up with 65890 (maybe trig 5 but I didn't really test it here?)

t_keystrokes = events[events[:,2]==65282]
t_mute = events[events[:,2]==65284]
t_unmute = events[events[:,2]==65288]


In [75]:
#cleaning the events (this can be combined with the cell above later
tc_keystrokes = clean_triggers(t_keystrokes)
tc_mute = clean_triggers(t_mute)
tc_unmute = clean_triggers(t_unmute)

#maybe need to reconcatenate and resort this?

In [76]:
#gathering the segments that are muted or unmuted
mute_raw = concat_uniform(raw, tc_mute, 30)
#unmute_raw = concat_uniform(raw, tc_unmute, 10)


17777
91512
173419


In [77]:
#plotting the keystroke events on segmented data
mute_raw.plot(events = tc_keystrokes)

Using pyopengl with version 3.1.6


Possible analyses

In [ ]:
#epoching by keystroke

#correlation to spectrogram (mTRF?)